**OLD: TRYING TO EXTRACT TOPICS FROM PROJECTS AND CATEGORIZING AUTOMATICALLY**

In [ ]:
!pip install PyGithub

In [ ]:
import json
from github import Github
from github import Auth

In [ ]:
# you can store the GitHub token in a file called token.txt
file_path = 'C:\\Users\\alenk\\Desktop\\TU_e\\2_Empirical_Methods_in_SE\\token.txt'

try:
    with open(file_path, 'r') as file:
        token = file.read()
except FileNotFoundError:
    print(f"File not found at the specified location: {file_path}")

In [ ]:
auth = Auth.Token(token)
g = Github(auth=auth)
g.get_user().login

In [ ]:
# defined the dictionary of the systems to be analyzed
with open('./groups/systems_list.json') as json_file:
    systems = json.load(json_file)

In [ ]:
# a dictionary with number_nameofthesystem or owner as key and the repository name as value\n
repos_dict = {}

for i in range(len(systems)):
    values = systems.get(list(systems.keys())[i]).split('/')
    values[0] = str(i)+"_"+str(values[0])
    repos_dict[values[0]] = values[1]

In [ ]:
repos_dict

In [ ]:
headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {token}",
}

In [ ]:
import requests

# get the topics(tags) of each repository from the url
topics = {}

for key, value in repos_dict.items():
    keys = key.split('_')[1]
    url = f"https://api.github.com/repos/{keys}/{value}/topics"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        topics[key] = response.json().values()
    else:
        print(f"Error: {response.status_code}")
        print(url)

In [ ]:
topics

In [ ]:
print("found topics for {} repos out of {} initial systems".format(len(topics), len(repos_dict)))

Get the most frequent tags

In [ ]:
flat_list = []

for key, value in topics.items():
    flat_list.extend(value)

flattened_list = [element for sublist in flat_list for element in sublist]

In [ ]:
from collections import Counter

def top_frequent_elements(lst, num):
    counter = Counter(lst)
    most_common = counter.most_common(num)
    return most_common
    #top_elements = [item[0] for item in most_common]
    #return top_elements

In [ ]:
frequency_rank = 15
result = top_frequent_elements(flattened_list, frequency_rank)
result

Clustering of text from the dictionary of projects:tags

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

In [ ]:
class FlattenedListIterable:
    def __init__(self, input_list):
        self.input_list = input_list
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index < len(self.input_list):
            value = self.input_list[self.index]
            self.index += 1
            return str(value)
        else:
            raise StopIteration

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, max_features=1000, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(FlattenedListIterable(flattened_list))

In [ ]:
lsa = TruncatedSVD(n_components=2)
lsa_matrix = lsa.fit_transform(tfidf_matrix)

In [ ]:
k = 10
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(lsa_matrix)

In [ ]:
labels = kmeans.labels_
cluster_centers = kmeans.cluster_centers_

plt.figure(figsize=(10, 8))
plt.scatter(lsa_matrix[:, 0], lsa_matrix[:, 1], c=labels, cmap='viridis')
plt.scatter(cluster_centers[:, 0], cluster_centers[:, 1,], s=200, c='red')
plt.title("Text Clustering with K-Means")
plt.show()

In [ ]:
initial_words = tfidf_vectorizer.inverse_transform(lsa.inverse_transform(cluster_centers))
initial_words